In [1]:
from functions import *

c:\Users\Gebruiker\anaconda3\envs\env_project_final\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
df_reviews = pd.read_csv('../../data/cleaned/reviews.csv', encoding='utf-8', parse_dates=['creation_timestamp','answer_timestamp'] )
df_reviews = df_reviews.copy()

In [ ]:
# Download Portuguese stopwords (ex: "o", "a", "os", "as", "de", "que", "em".)

setup_nltk()
portuguese_stopwords = get_portuguese_stopwords()

In [ ]:
df_reviews['cleaned_review'], df_reviews['emojis'] = zip(*df_reviews['message'].apply(lambda x: clean_text(x, portuguese_stopwords)))

In [ ]:
emoji_counts = df_reviews['emojis'].apply(lambda x: [c for c in x]).explode().value_counts()

In [ ]:
# Visualize emoji distribution

plt.figure(figsize=(12, 6))
emoji_counts.head(10).plot(kind='bar')
plt.title('Top 10 Most Used Emojis in Reviews')
plt.xlabel('Emoji')
plt.ylabel('Count')
plt.show()

In [ ]:
# Get most common words
all_words = ' '.join(df_reviews['cleaned_review']).split()
valid_words = [word for word in all_words if is_valid_word(word)]
word_counts = Counter(valid_words)
top_words_pt = word_counts.most_common(20)

In [ ]:
top_words_pt = word_counts.most_common(20)
top_words_en = [(translate_to_english(word), count) for word, count in top_words_pt]

In [ ]:

# Emotion Detection with Hugging Face
from transformers import pipeline

# Load multilingual emotion classification model
emotion_classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", top_k=None)

# Function to get emotions
def get_emotions(text):
    results = emotion_classifier(text)
    return {r['label']: r['score'] for r in results[0]}

# Apply to a sample of reviews
sample_df = df.sample(1000)
sample_df['emotions'] = sample_df['cleaned_review'].apply(get_emotions)

# Analyze results
sample_df['top_emotion'] = sample_df['emotions'].apply(lambda x: max(x, key=x.get))
sample_df['top_emotion'].value_counts().plot(kind='bar')
plt.title('Distribution of Top Emotions')
plt.show()

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-xlm-roberta-base-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-xlm-roberta-base-sentiment")

In [ ]:
from transformers import pipeline
model_path = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
sentiment_task = pipeline("sentiment-analysis", model=model_path, tokenizer=model_path)
sentiment_task("T'estimo!")
